In [2]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Masking, Embedding, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn import metrics
from sklearn import linear_model
from keras.models import Sequential
import pickle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import requests
import json
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split



nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
nltk.download('stopwords')

from scipy import linalg
from collections import defaultdict

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Importation des données

In [30]:
data = pd.read_csv("gdrive/MyDrive/projetNLP/DataNLP.csv",encoding = "ISO-8859-1",header =None,names= ['sentiment','id','date','flag','user','text'])
data=data[798000:802000]
N=len(data)
print(N)
print(data)

12000
        sentiment  ...                                               text
790000          0  ...                   Virpi's last day.  Busy as tits.
790001          0  ...  @SherilynMoon unfortunate then that I have no ...
790002          0  ...  just dropped off the car to get the brakes loo...
790003          0  ...  Colin Kelly on Clyde1 Should get One Republic ...
790004          0  ...  just a little upset i had a chance to take 4 o...
...           ...  ...                                                ...
801995          4  ...  @FizzyDuck Five? Seems a little bit too late i...
801996          4  ...  ryanodonnell: @AttractMode Thanks for putting ...
801997          4  ...                      @damygeebo Carli's my friend 
801998          4  ...  I feel so great for starting twitter at suzann...
801999          4  ...  @Luke_Stephens: I just said I was wondering ab...

[12000 rows x 6 columns]


In [12]:
# echantillon= data[:100000]
# def minuscule(echantillon):
#   for i in range(len(echantillon)):
#     echantillon[i]=echantillon[i].lower()
  
#   return echantillon
# print(minuscule)

# Tokenization

In [13]:

#On utilise une fonction de Keras qui permet de vectoriser un corpus de texte.
tokenizer = Tokenizer(num_words=20000, split=" ")

tokenizer.fit_on_texts(data["text"].values)

X= tokenizer.texts_to_sequences(data["text"].values)

X = pad_sequences(X)
print(X[:33])
print(X.shape)

[[   0    0    0 ...  361   30  160]
 [   0    0    0 ...  304  771 1878]
 [   0    0    0 ... 1396    5  144]
 ...
 [   0    0    0 ...  306  874   18]
 [   0    0    0 ... 1896  983 3068]
 [   0    0    0 ...   90   46   43]]
(4000, 32)


In [14]:
# 4 devient [0 1] pour les positifs 
# 0 devient [1 0] pour les négatifs
Y=pd.get_dummies(data["sentiment"]).values
print(Y)
print(Y.shape)

[[1 0]
 [1 0]
 [1 0]
 ...
 [0 1]
 [0 1]
 [0 1]]
(4000, 2)


# Modèle RNN

In [15]:
model = Sequential()

#Transforme les entiers positifs (index) en vecteurs de taille fixe.
#encodage
model.add(Embedding(20000, 256, input_length=X.shape[1]))
model.add(Dropout(0.3))


model.add(LSTM(256,return_sequences=False,dropout=0.3,recurrent_dropout=0.2))

# Le Sigmoïde produit une courbe en forme de S.

model.add(Dense(2, activation="sigmoid"))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

#on entraine la data avec 70% des données afin de pouvoir tester avec le reste
x_train,x_test,y_train,y_test = train_test_split(X,Y,train_size=0.7,random_state=0)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 256)           5120000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 5,645,826
Trainable params: 5,645,826
Non-trainable params: 0
_________________________________________________________________


# Entrainement

In [16]:
model.fit(x_train,  y_train, 
                   epochs=8, batch_size=100,verbose =2)

Epoch 1/8
28/28 - 19s - loss: 0.6434 - accuracy: 0.6304
Epoch 2/8
28/28 - 14s - loss: 0.3960 - accuracy: 0.8368
Epoch 3/8
28/28 - 14s - loss: 0.2095 - accuracy: 0.9207
Epoch 4/8
28/28 - 14s - loss: 0.1058 - accuracy: 0.9657
Epoch 5/8
28/28 - 14s - loss: 0.0729 - accuracy: 0.9764
Epoch 6/8
28/28 - 14s - loss: 0.0304 - accuracy: 0.9918
Epoch 7/8
28/28 - 14s - loss: 0.0191 - accuracy: 0.9946
Epoch 8/8
28/28 - 14s - loss: 0.0083 - accuracy: 0.9986


#Evaluation

In [17]:
model.evaluate(x_test,y_test)

38/38 [==============================] - 2s 42ms/step - loss: 1.0228 - accuracy: 0.7675


[1.0227912664413452, 0.7674999833106995]

# Sauvegarde du modèle

In [ ]:
model.save('/content/gdrive/MyDrive/ProjetNLP/Projet_NLP/Model_1')

# Utilisation de l'API Yelp

In [18]:
api_key='q1HJoEmRMdypie_E6lYwiih0hNfuSatSTQXH6Bh9fnltm30JX4XO1TSCd6NARRVszrs4lMXb7N3i6LWB_UIBEinyG_ah-0expG6GUnzTdUss2gvNQRcJ7fJhK5g2YHYx'
headers = {'Authorization': 'Bearer %s' % api_key}


url = "https://api.yelp.com/v3/businesses/FEVQpbOPOwAPNIgO7D3xxw/reviews"
req = requests.get(url, headers=headers)
print('the status code is {}'.format(req.status_code))


data=json.loads(req.text)
data

the status code is 200


{'possible_languages': ['fr', 'en', 'zh', 'pt', 'de', 'it', 'sv', 'ja', 'es'],
 'reviews': [{'id': 'qOiuCTz_Gzgmose1GHaAlg',
   'rating': 5,
   'text': "Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...",
   'time_created': '2020-12-19 07:52:35',
   'url': 'https://www.yelp.com/biz/shake-shack-new-york-2?adjust_creative=KzZVmcfbYqCMY9omoha5NA&hrid=qOiuCTz_Gzgmose1GHaAlg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KzZVmcfbYqCMY9omoha5NA',
   'user': {'id': 'FRKnLBnlFiasr1Dc8oOIGQ',
    'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/1wm0yjWSw92j_ZsUFZBGzQ/o.jpg',
    'name': 'Sarah G.',
    'profile_url': 'https://www.yelp.com/user_details?userid=FRKnLBnlFiasr1Dc8oOIGQ'}},
  {'id': '7d56A_ObMPHHyywcfhnrUw',
   'rating': 5,
   'text': 'Happened to be in the city today and not too far from here so I had to stop by and pick up cheese fries (I love those c

# Traitement des futures données

In [28]:
def data_nopunct(data):
    punct = string.punctuation
    tab_punct=[]
    for i in range(len(punct)):
      tab_punct.append(punct[i])


    no_punct_dataset=[]
    element=""
  
    for letter in data:
      if letter not in tab_punct and not letter.isdigit():
        element += letter

    no_punct_dataset.append(element)
    element =""
    
    return no_punct_dataset
    
def traitement(my_string):
    
  dataset_string=data_nopunct(my_string)

  tab_string=pd.DataFrame(dataset_string, columns = ["text"])

  
  tokenizer = Tokenizer(num_words=20000, split=" ")
  tokenizer.fit_on_texts(tab_string["text"].values)

  X_string= tokenizer.texts_to_sequences(tab_string["text"].values)
  X_string = pad_sequences(X_string)

  return X_string


# Prediction

In [29]:
#ph1 =data["reviews"][0]["text"]
ph2="If humanity doesn't merge with AI we are doomed. If we do we lose our humanity."
ph3="I think it's part of the future, but i don't think it will enable us to keep up with AI. Our meat brain is never gonna be on par with an AGI. The only thing we can do it's hold it back until we know how to control it"


#test1 = traitement(ph1)
test2 = traitement(ph2)
test3 = traitement(ph3)


#preds1 = model.predict(test1)
preds2 = model.predict(test2)
preds3 = model.predict(test3)
#print(preds1)
print(preds2)
print(preds3)


pred_list=[preds1,preds2,preds3]
sentiment_list=["positif","negatif","positif"]

for i in range(len(pred_list)):
  if pred_list[i][0][0] < (1/3):
    print('predicted sentiment : negatif')
    print(f"target sentiment : {sentiment_list[i]}")
  elif  pred_list[i][0][0] > (2/3): 
    print('predicted sentiment : positif')
    print(f"target sentiment : {sentiment_list[i]}")
  else:
    print('predicted sentiment : neutre')
    print(f"target sentiment : {sentiment_list[i]}")


[[0.13807243 0.86394286]]
[[9.992230e-01 7.583499e-04]]
predicted sentiment : positif
target sentiment : positif
predicted sentiment : negatif
target sentiment : negatif
predicted sentiment : positif
target sentiment : positif
